# Using vireo for multiplexing, followed by lineage tracing of those groups

# 1. Set up

In [1]:
import os
os.chdir("/data2/mito_lineage/")#Analysis/multiplex")

In [2]:
# papermill --cwd /data2/mito_lineage -p INDIR {INDIR} -p OUTDIR data/processed/mttrace/CHIP_april08_2021_Croker/MTblacklist/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/clones -p  src/vireo/2_MT_Lineage_Construct.ipynb 

In [3]:
pwd

'/data2/mito_lineage'

### Remove this when running in snakemake

## 1.1 Parameters

In [4]:
# INDIR=""
# OUTDIR=""
# N_DONORS=5
INDIR= "data/processed/mttrace/CHIP_april08_2021_Croker/MTblacklist/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/multiplex"
OUTDIR= "data/processed/mttrace/CHIP_april08_2021_Croker/MTblacklist/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/clones"
N_DONORS=5
DEBUG=False

In [5]:
# # Parameters
# #INDIR="data/CHIP_april08_2021/MTblacklist/chrM/pseudo/minC200_minAF0.0001/numC25000_ispropFalse/"
# INDIR=""
# OUTDIR=""
# #OUTDIR="data/CHIP_april08_2021/MTblacklist/chrM/pseudo/minC200_minAF0.0001/numC25000_ispropFalse/lineages"
# N_DONORS=5

## 1.2 Import packages

In [14]:
from os.path import join, exists, dirname
from icecream import ic
from glob import glob
import mplh.cluster_help as ch
import os
from vireoSNP.plot.base_plot import heat_matrix
from vireoSNP import Vireo
import src.pseudo_batch as pb
import vireoSNP
ic(vireoSNP.__version__)
from tqdm import tqdm
import numpy as np
from scipy import sparse
from scipy.io import mmread
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from src.vireo.vireo_utils import run_vireo, plot_vireo_out, extract_clusters, run_elbo, separate_donors
%load_ext autoreload
%autoreload 2
from src.vireo.vireo_utils import run_lineage
#np.set_icoptions(formatter={'float': lambda x: format(x, '.5f')})

0.4.2
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
if not DEBUG:
    ic.disable()


# 2 Lineage construction for each donor

In [9]:
# num_cores = 12# N_DONORS #multiprocessing.cpu_count()
# inputs = tqdm(np.arange(N_DONORS))
# for n in np.arange(N_DONORS):
#     ic('n', n)
#     run_lineage(n, INDIR, OUTDIR, n_clone_list=[5, 10, 20, 40, 100], rerun_model=False)

In [10]:
n_clone_list=[20, 40, 100]

In [11]:
cells_meta = pd.read_csv(join(INDIR, f"cells_meta.tsv"), sep='\t')

cells_meta=cells_meta.astype({"donor_index": "Int64", "donor": "Int64"})
cells_meta

,new index,ID,raw ID,condition,donor,donor_index
0,1,AAACGAATCTTACTCA-1_Input,AAACGAATCTTACTCA-1,Input,0,1
1,2,AAAGGATAGTCGTGAG-1_Input,AAAGGATAGTCGTGAG-1,Input,2,1
2,3,AAATGAGGTCAATACC-1_Input,AAATGAGGTCAATACC-1,Input,NaN,NaN
3,4,AAATGCCAGGAACCGT-1_Input,AAATGCCAGGAACCGT-1,Input,4,1
4,5,AAATGCCCACCGAAAG-1_Input,AAATGCCCACCGAAAG-1,Input,NaN,NaN
...,...,...,...,...,...,...
4793,4794,TTTGTGTCATTCTTTG-1_Flt3l,TTTGTGTCATTCTTTG-1,Flt3l,0,1136
4794,4795,TTTGTGTTCAAGAGGC-1_Flt3l,TTTGTGTTCAAGAGGC-1,Flt3l,0,1137
4795,4796,TTTGTGTTCGCAGATT-1_Flt3l,TTTGTGTTCGCAGATT-1,Flt3l,1,788
4796,4797,TTTGTGTTCGCTATAG-1_Flt3l,TTTGTGTTCGCTATAG-1,Flt3l,3,949


In [12]:
def run_lineage_wrap(indir, outdir, n_donors, n_clone_list, cells_meta):
    for clone in n_clone_list:
        cells_meta_col = f"lineage{clone}"
        curr_outdir = join(outdir, cells_meta_col)
        if not exists(curr_outdir):
            os.mkdir(curr_outdir)
        curr_cells_meta = cells_meta.copy()
        all_cells_meta = [] #concat across donors
        
        for donor in np.arange(n_donors):
            print(f'clone: {n_clone_list} donor: {donor}')
            curr_ad_f = join(indir, f"donor{donor}.AD.mtx")
            curr_dp_f = join(indir, f"donor{donor}.DP.mtx")
            curr_labels_f = join(indir, f"donor{donor}.labels.txt")
            curr_vcf_f = join(indir, f"donor{donor}.vcf")
            curr_labels_rawID = join(indir, f"cell_labels.donor{donor}.txt")
            prefix=f"donor{donor}_"
            ic(curr_ad_f)
            ic(curr_dp_f)
            curr_ad = mmread(curr_ad_f).tocsc()
            curr_dp = mmread(curr_dp_f).tocsc()
            curr_labels = pd.read_csv(curr_labels_f, index_col=0)
            curr_vcf = pd.read_csv(curr_vcf_f, sep='\t')
            ic('curr_vcf')
            ic(curr_vcf.head())
            #curr_cells_meta = pd.read_csv(curr_labels_rawID, index_col=0)
            ic('curr_labels')
            ic(curr_labels.head())
            prefix = f"donor{donor}_"
            model_name = f"{prefix}lineage_{clone}"
            modelCA, elbo = run_vireo(curr_ad, curr_dp, clone, n_cores=32, plot_qc=True,out_f=join(curr_outdir, model_name))
            try:
                doublet_prob = modelCA.predict_doublet(AD, DP, update_GT=False, update_ID=False)[0].sum(axis=1)
            except AttributeError: # New version of Vireo 2021
                #doublet_prob = vireoSNP.utils.vireo_doublet.predict_doublet(modelCA,AD, DP, update_GT=False, update_ID=False)[0].sum(axis=1)
                doublet_prob = vireoSNP.utils.vireo_doublet.predict_doublet(modelCA, curr_ad, curr_dp, update_GT=False, update_ID=False)[0].sum(axis=1)
            

            separate_donors(curr_ad, curr_dp, modelCA, curr_cells_meta, curr_outdir,
                            doublet_prob,
                            prob_thresh=0.9, doublet_thresh=0.9,
                            cells_ind_col='donor_index',
                            out_name="lineage", prefix=prefix, cells_filt_col="donor", 
                            cells_filt_val=donor, vars_meta=curr_vcf)
            all_cells_meta.append(pd.read_csv(join(curr_outdir, f"{prefix}cells_meta.tsv"), sep='\t'))

        # Concat cells_meta across donors
        all_cells_meta = pd.concat(all_cells_meta, axis=0)
        all_cells_meta = all_cells_meta.astype({"lineage_index": "Int64", "lineage": "Int64"})
        ic('all_cells_meta', all_cells_meta.head())
        all_cells_meta.to_csv(join(curr_outdir, "cells_meta.tsv"), sep='\t')
    return

In [13]:
run_lineage_wrap(INDIR, OUTDIR, N_DONORS, n_clone_list, cells_meta)

data/processed/mttrace/CHIP_april08_2021_Croker/MTblacklist/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/multiplex/donor0.AD.mtx
data/processed/mttrace/CHIP_april08_2021_Croker/MTblacklist/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/multiplex/donor0.DP.mtx
curr_vcf
  index_x #CHROM    POS REF_x ALT  strand_correlation_x     vmr_x  \
0     NaN   chrM  16512   NaN   C                   NaN       NaN   
1   73A>G   chrM     73   73A   G              0.996919  0.382955   
2  146T>C   chrM    146  146T   C              0.993106  0.591151   
3  150C>T   chrM    150  150C   T              0.979327  0.670372   
4  195T>C   chrM    195  195T   C              0.990257  0.623396   

   n_cells_over_5_x  n_cells_over_20_x  \
0               NaN                NaN   
1             119.0              116.0   
2               2.0                1.0   
3           

[ -1   1   2   3   4   5   7   8   9  10  12  13  14  17  19  20  22  26
  28  29  32  33  35  38  39  40  41  43  44  45  46  48  49  50  51  53
  54  56  57  58  60  62  63  64  67  68  74  77  79  80  81  82  84  87
  88  89  90  91  93  96  97 100 101 103 105 106 108 110 111 112 114 115
 116 118 119 120 123 125 126 129 130 131 132 134 136 138 139 143 145 147
 148 149 150 151 152 153 155 156 157 158 160 161 162 163 165 166 167 168
 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 188 190 192
 197 198 209 212 213 214 215 216 217 223 226 228 230 234 238 239 240 241
 243 245 246 247 249 250 253 257 258 261 262 264 266 268 269 271 273 280
 283 284 285 286 291 292 296 298 299 301 302 304 306 310 312 313 314 315
 318 321 334 336 339 341 342 343 345 346 347 354 357 358 360 362 363 365
 370 372 375 376 379 380 381 384 386 387 390 393 395 397 398 400 402 403
 406 407 410 412 413 414 419 421 423 424 429 430 432 433 435 436 445 447
 450 451 458 461]
lineage 2: 21 cells 
(1202, 3)
(1

curr_ad     Position  Cell  Count
0          1     1    635
6          6     1    260
10         8     1    852
13        10     1     64
14        11     1     18
donor0_lineage9
[ -1   0   1   2   3   4   5   8   9  10  12  13  14  17  20  21  22  25
  26  28  29  31  34  35  38  39  40  41  44  45  46  47  48  49  50  51
  52  53  54  56  57  58  61  62  64  65  67  68  70  71  74  76  77  79
  80  81  82  84  87  88  89  91  99 100 101 103 104 105 106 107 108 109
 110 112 114 116 117 118 119 121 123 124 125 126 127 129 130 131 132 138
 139 140 142 143 144 145 147 148 149 150 152 153 155 157 160 161 162 163
 165 166 167 168 169 171 172 173 174 175 176 177 178 179 180 181 182 183
 188 190 193 198 201 209 212 215 217 219 220 222 226 229 230 233 235 239
 241 244 245 246 247 248 249 250 253 255 256 257 258 261 264 266 267 268
 269 271 272 275 279 284 285 289 292 293 295 296 297 298 299 301 302 303
 304 305 306 310 312 313 315 318 321 329 330 334 336 337 338 341 343 345
 346 347 354 355 

[ -1   1   2   3   5   8   9  10  12  13  14  17  25  26  35  38  39  41
  46  48  49  50  53  54  56  57  58  62  63  67  68  74  77  78  79  80
  81  87  88  90  91 100 101 103 105 106 107 108 110 112 114 116 117 118
 119 123 125 126 129 130 131 132 138 139 145 147 148 149 150 152 153 155
 157 160 161 162 163 165 166 168 169 170 171 172 175 176 177 178 179 180
 181 182 183 188 190 212 215 226 238 239 247 249 253 258 261 262 264 266
 275 278 282 284 285 286 299 302 307 312 314 317 322 323 329 331 334 337
 339 341 343 351 352 357 362 363 367 370 372 380 386 395 397 398 400 406
 407 408 410 413 415 421 423 426 432 433 444 453]
lineage 18: 9 cells 
(539, 3)
(539, 3)
curr_ad     Position  Cell  Count
0          1     1    168
4          4     1      5
6          5     1     70
10         7     1    151
13         9     1      3
donor0_lineage18
[ -1   0   2   3   4   5   6   8   9  12  13  14  15  17  25  26  28  31
  33  35  37  38  39  40  41  42  45  46  48  49  53  54  57  58  62  63


cell meta
             new index                        ID              raw ID  \
donor_index                                                            
1                   11  AAGGTTCGTTCCAGTA-1_Input  AAGGTTCGTTCCAGTA-1   
2                   16  ACAGCGCCATAGCAGG-1_Input  ACAGCGCCATAGCAGG-1   
3                   31  AGCGTGCAGAACTAAC-1_Input  AGCGTGCAGAACTAAC-1   
4                   34  AGGCGTCAGAACAGGA-1_Input  AGGCGTCAGAACAGGA-1   
5                   46  ATGTTTCTCTCGACCT-1_Input  ATGTTTCTCTCGACCT-1   

            condition  donor  lineage  lineage_index  
donor_index                                           
1               Input      1      NaN            NaN  
2               Input      1      NaN            NaN  
3               Input      1      NaN            NaN  
4               Input      1      NaN            NaN  
5               Input      1      NaN            NaN  
[ -1   0   1   2   5   7   8   9  10  12  15  16  19  23  24  26  27  31
  32  33  37  38  39  40  4

...

curr_ad      Position  Cell  Count
6           1     1    196
36          5     1      8
41          6     1    484
60          9     1      5
114        14     1    554
donor4_lineage28
[  5   9  40  49  54  76  89 113 137 152 157 160 164 167 172 186 207 214
 241 271 278 321 365 391 407 448]
lineage 29: 1 cells 
(26, 3)
(26, 3)
curr_ad     Position  Cell  Count
6          1     1    109
10         2     1      4
41         3     1    278
50         4     1      6
55         5     1      6
donor4_lineage29
[ -1   0   2   5   8   9  10  11  13  16  17  25  27  28  32  35  39  40
  41  49  50  51  55  57  58  59  61  63  64  69  70  76  78  79  81  82
  88  89  91  92 100 101 102 106 107 109 111 113 115 116 117 118 122 124
 128 130 137 138 145 147 148 149 152 153 157 159 160 162 164 167 168 169
 170 172 173 174 175 176 177 178 179 180 186 193 194 195 197 198 199 204
 207 208 209 210 211 212 215 216 219 220 222 223 229 233 236 237 240 242
 248 252 255 257 259 264 278 281 284 292 299 301 3

83        24     1      3
donor4_lineage49
[  5  11  40  41  59  79 113 122 138 152 157 167 172 186 188 206 221]
lineage 50: 1 cells 
(17, 3)
(17, 3)
curr_ad     Position  Cell  Count
6          1     1    397
12         2     1      4
41         3     1   1044
42         4     1     31
60         5     1      2
donor4_lineage50
[ -1   1   5   8   9  10  11  13  25  27  29  35  40  41  42  47  49  51
  55  57  59  63  69  70  76  79  82 101 102 109 111 113 115 116 117 118
 122 124 125 128 129 131 137 138 147 148 149 150 152 153 157 159 163 166
 167 168 170 172 173 176 178 179 180 181 186 188 194 202 204 207 211 215
 220 224 229 233 234 236 241 243 245 251 252 255 257 260 266 272 278 281
 282 284 286 296 297 299 315 321 329 331 336 341 347 350 352 357 359 365
 368 369 370 376 377 380 390 391 392 396 400 401 402 406 408 412 421 423
 439 440 448]
lineage 51: 8 cells 
(259, 3)
(259, 3)
curr_ad     Position  Cell  Count
6          3     1    442
12         7     1      2
14         8     1 

8         8     1     21
donor4_lineage66
[ -1   5   8  10  13  27  28  29  35  40  42  47  51  57  58  59  64  69
  70  76  81  88  89  92 100 101 106 107 109 113 115 116 117 118 122 124
 128 129 130 131 137 138 145 148 149 152 153 157 159 160 162 163 166 167
 168 170 172 173 174 175 177 181 186 194 204 210 211 220 224 229 233 236
 242 243 245 249 252 257 260 272 274 278 284 290 295 299 302 303 307 308
 321 324 329 330 331 341 347 350 357 360 365 366 369 370 371 376 380 384
 386 391 393 399 400 401 402 404 406 407 410 416 419 432 433 441 447]
lineage 67: 8 cells 
(243, 3)
(243, 3)
curr_ad     Position  Cell  Count
0          1     1      2
6          2     1    126
14         5     1      2
36         9     1      4
41        10     1    289
donor4_lineage67
[ -1   2   4   5   8   9  11  13  15  16  19  25  27  28  29  32  35  36
  40  41  42  44  47  49  50  51  54  55  57  58  59  60  62  63  64  67
  69  70  72  76  78  79  81  82  86  88  89  91  92 100 101 102 104 105
 106 108 10

[  5  13  27  40  42  50  54  76 106 109 111 113 122 138 152 153 155 157
 159 167 168 170 172 186 207 252 257 265 282 286 299 310 321 329 331 364
 370 376 380 406 427 443]
lineage 83: 2 cells 
(52, 3)
(52, 3)
curr_ad      Position  Cell  Count
6           1     1     38
41          4     1    168
107         9     1      2
110        10     1      2
114        12     1    217
donor4_lineage83
[ -1   5   8   9  10  13  35  40  42  49  50  57  64  69  76  78  81  92
 107 113 115 116 117 122 128 129 130 131 137 138 145 147 148 149 152 153
 155 157 160 161 163 167 170 172 174 175 177 178 179 186 204 207 210 220
 233 242 243 262 269 272 274 278 282 284 302 308 321 327 331 336 347 350
 352 353 365 369 370 376 391 393 396 397 406 407 410 422 423]
lineage 84: 6 cells 
(160, 3)
(160, 3)
curr_ad      Position  Cell  Count
6           2     1     70
41          8     1    103
43          9     1      2
114        20     1    162
123        24     1     10
donor4_lineage84
[  5   8  10  25  35  39

[]
lineage 99: 0 cells 
(0, 3)
(0, 3)
curr_ad Empty DataFrame
Columns: [Position, Cell, Count]
Index: []
donor4_lineage99
cell meta
             new index                        ID              raw ID  \
donor_index                                                            
1                    4  AAATGCCAGGAACCGT-1_Input  AAATGCCAGGAACCGT-1   
2                    6  AACAAAGCAATTCAGC-1_Input  AACAAAGCAATTCAGC-1   
3                   12  AATACGCGTCATCGTA-1_Input  AATACGCGTCATCGTA-1   
4                   27  ACTAGGTGTATTCACG-1_Input  ACTAGGTGTATTCACG-1   
5                   40  ATCCTGCAGACTAATG-1_Input  ATCCTGCAGACTAATG-1   

            condition  donor  lineage  lineage_index  
donor_index                                           
1               Input      4       22              1  
2               Input      4       62              1  
3               Input      4       47              1  
4               Input      4      NaN            NaN  
5               Input      4     